## I. 系统架构
#### 系统架构演化
* 垂直架构：MVC，一个大而全的项目，放在一个服务器里
* RPC：远程过程调用，把不同的部分，业务逻辑，数据存储拆分到不同的服务器，通过网络远程协议调用
    * 缺点：这些过程没有中心化管理，很难监控和管理
        * 协议栈
* SOA: RPC + 服务治理中心
* 微服务： 在SOA的基础上，又把服务器虚拟化，拆分到一个功能一个服务。e.g 利用docker
![HBASE Map Reduce](img/rpc.png)
##### 问题
* 这些架构都没有提供高可用性，服务器挂了怎么办？
* 提供高可用性，那么意味着一个服务有多个节点提供，怎么调度？怎么把服务挂起？在服务中路由？
    * solution：
        * 集群
        * 负载均衡/反向代理(proxy)： nginx
        * 调度算法：random robin，轮询，hash-ip, ...
        
##### 负载均衡

        
##### RPC
Remote procedure call, 远程服务调用，允许像调用本地服务一样调用远程服务。
###### 提供
* 通信机制，TCP/UDP
* 序列化方式，XML/JSON/二进制
* 通信细节

###### 不提供
* 服务治理
* 服务发现
* 就是两个进程之间相互调用

###### 实现
* 基于消息队列系统的实现，Rabbitmq/kafka
* 基于java NIO, Apache Thrift
* 基于序列化框架的实现，Avro RPC

##### SOA
* RPC + 服务治理中心
* 利用zookeeper来提供治理

###### Zookeeper
* 调度中心
* clients 告诉 zookeeper需要什么服务，自己的地址
* zookeeper 查询服务，告诉client， cients自己去找service
* 阿里提供dubbo 提供SOA服务治理框架

##### zookeeper + dubbo
https://www.bilibili.com/video/av27237493/?p=4

###### 分布式引发的问题
* 分开到不同的服务器 -> 网络通信
* 怎么找到需要的服务器地址，**动态寻址**，通信 => 服务的注册和发现

## II. 网络编程
### 2.1 协议
* TCP
    * 三次握手，四次挥手
* HTTP
* AMQP
* SOAP

#### 连接分类
* 长连接
    * client和server之间一只保持连接，通过心跳检测来保证连接有效
* 短连接
    * client发完请求，收到回复就断开

##### 可靠性保证
* 链路有效性检测
    * tcp层面的heartbeat检测
    * 协议层的heartbeat检测，e.g SMPP proxy
    * 应用层的heartbeat检测
* 断连重连机制
    * 链路失败时，一定要释放资源 handler（句柄）
    * client等待一段时间后尝试重新连接，（expenencial backoff）
* 消息缓存重发，queue

### 2.2 Socket 编程
#### 2.2.1 fileno 文件描述符

### 2.3 异步编程
#### 协程 - 轻量级线程

#### 线程

#### 进程

#### 多机器 - 网络


### 2.4 网络通信框架
* blocking io/BIO
* Not blocking io/NIO
* Async IO/AIO

### 2.5 负载均衡 load balancing
区别是client知不知道地址
#### 2.5.0 概念
* 正向代理：知道你要去哪儿 vpn， google.com -> go
* 反向代理：到了一个中心，不用关心去哪儿，我说要什么服务，中心替我找
* session, cookie:
    * 两个节点中，session不能共享，很多人喜欢在session中存数据，e.g 登陆验证。在节点中切换中，登陆不停失效
    * solution
        * A.不要让一个客户的请求在不同的节点中跳转, 依靠调度算法
        * B.session共享，session在集群中同步，不推荐, 因为session会占用链接，而每个服务器的请求的数目是有限制的，session共享严重限制了可以处理的请求数。 共享数据要占用一个链接，而且同步还消耗性能，性能反而遍地
* 调度算法
    * 轮循算法：每个请求就不听跳转 + redis 可以解决
    * hash-ip：一致性哈希算法

        
### 2.6 编码与序列化
#### 序列化
* parquet
* Avro
* Thrift
* PB
* MessagePack

## III. 分布式解决方案
### 3.1 数据一致性
#### 3.1.0 原理
##### 1) ACID
* Atomicity 原子性
    * 全部成功执行
    * 全部不执行
* Consistency 一致性
    * 数据强一致性
* Isolation 隔离性
    * 在并发环境中，事务时相互隔离的，事物间不能相互干扰
    * 隔离级别
        * Read Uncomitted, 允许dirty read
        * Read committed
        * Repeatable Read，不允许dirty read
        * Serializable，最严格的隔离级别，并发和串行结果完全一样
* Durability 持久性
    * 事务一旦提交，对数据的变更就是永久的
   
![HBASE Map Reduce](img/isolation.png) 
##### 2)  CAP
* 一个系统不能同时满足Consistency，Availability and Partition tolerance
* 最多只能满足其二
* 分区容错性：
    * 分布式系统中，在任何网络分区有故障时，仍然能够提供满足一致性和可用性的服务
* 三个特性中
    * P是必须有的，不然就不是分布式系统啦 
    * C可以放弃强一致性，而追求最终一致性
    * 架构师往往组要根据业务特点在C和A中间需求平衡
    
#####  3) BASE
* Basically Available 基本可用
    * 响应时间的损失
    * 功能上的损失
    * 在峰值访问量，通过增加时延，和减少额外的功能来换取系统性能
* Soft state 软状态
    * 允许数据存在中间状态
    * 允许不同节点在同步数据副本是存在不一致的状态和延时
* Eventually consistent 最终一致性
    * 经过一定的时间窗口后，数据一致，不要求实时保证系统的强一致性
        * 因果一致性 Causal consistency
            * 如果进程A在改变数据后通知进程B，则需要保证，其之后的更新都通知B
            * 如果没有这种关系的进程A,C 则不需要
        * read your writes
        * session consistency
            * 同一个session的数据读写，总保持一致
        * Monotonic read consistency
        * Monotonic write consistency

##### 4) 一致性协议
* 2PC - 两阶段提交
* 3PC - 三阶段提交
* Paxos

#### 3.1.1 Paxos 
https://www.bilibili.com/video/av21667358?from=search&seid=2794461600340825245
* 其实是一个共识算法
* 一致性: S3 等产品保证 100% CP，99.99 A

###### 一致性模型
* 弱一致性：
    * 最终一致性
        * DNS
        * Gossip (Cassandra的通信协议)
* 强一致性
    * 同步
    * paxos
    * raft(multi-paxos)
    * ZAB(multi-paxos) -- zookeeper的一致性模型

###### 强一致性算法
* 主从同步复制
    * 主从通过复制保持一致 -- 很难高可用
    * 多数派 >n/2 
        * 在并发环境下，无法保证正确性
    * paxos
        * basic paxos
        * multi paxos
        * fast paxos

###### basic paxos
* 角色：
    * client
    * propser：接受client请求，向集群提出propose，冲突发生时调节，像议员
    * accpetor(voter): 提议投票和接收者，只有大于majority，提议才会接受，像国会
    * learner：提议接受者，backup，对集群一致性没有影响，像记录员
    
* phases：
    1. Prepare
    2. Promise
    3. Accept
    4. Accepted

* 问题：
    * 难实现
    * 效率低
    * 活锁
        * 如果在一个propose提出时，有另一个propose进来，靠id编号，或者时间取舍最近的
        * 那如果两个client不停的发最近的propose来竞争资源，那么就会造成活锁
    
![HBASE Map Reduce](img/basic_paxos.png)

###### 强一致性算法 -- Raft
* 划分成三个子问题
    * leader election
    * log replication
    * safety
* 重定义角色
    * leader
    * follower
    * candidate

#### 3.1.2 Raft

### 3.2 服务可用性和分区容错的架构

#### 3.2.1 master/slave

#### 3.2.2 active/standby

#### 3.2.3 leader/follower

### 3.3 分布式锁
#### 3.3.0 锁
* 悲观锁
* 乐观锁
* 死/活锁
* 读/写锁

#### 3.3.1 基于Chubby实现

#### 3.3.2 基于Zookeeper实现

#### 3.3.3 基于redis实现

### 3.4 缓存
#### 构建LRU Cache

#### 构建LFU Cache

### 3.5 分布式文件系统
* GFS
* HDFS

## IV.  消息队列
### 4.0 概念
#### 信道/channel
##### AMQP协议 - RabbitMQ
AMQP（高级消息队列协议）是一个异步消息传递所使用应用层协议规范，为面向消息中间件设计，基于此协议的客户端与消息中间件可以无视消息来源传递消息，不受客户端、消息中间件、不同的开发语言环境等条件的限制

* 端口：5672
* 优点
    * TCP 链接非常昂贵，而使用线程操作AMQP，在一个TCP链接中开通多个信道更优
    ![HBASE Map Reduce](img/AMQP.png)

AMQP当中有四个概念非常重要: 虚拟主机（virtual host），交换机（exchange），队列（queue）和绑定（binding）。一个虚拟主机持有一组交换机、队列和绑定。为什么需要多个虚拟主机呢？很简单，RabbitMQ当中，用户只能在虚拟主机的粒度进行权限控制。因此，如果需要禁止A组访问B组的交换机/队列/绑定，必须为A和B分别创 建一个虚拟主机。每一个RabbitMQ服务器都有一个默认的虚拟主机“/”。

Producer 要产生消息必须要创建一个 Exchange ，Exchange 用于转发消息，但是它不会做存储，如果没有 Queue bind 到 Exchange 的话，它会直接丢弃掉 Producer 发送过来的消息，当然如果消息总是发送过去就被直接丢弃那就没有什么意思了，一个 Consumer 想要接受消息的话，就要创建一个 Queue ，并把这个 Queue bind 到指定的 Exchange 上，然后 Exchange 会把消息转发到 Queue 那里，Queue 会负责存储消息，Consumer 可以通过主动 Pop 或者是 Subscribe 之后被动回调的方式来从 Queue 中取得消息。

#### 优点
* 应用解耦 - decouple 
    * queue作为客户端和服务器的中间件，使client和server应用解耦
* 支持高并发 - High Concurrency
    * 流量削峰 实际上就是**消息缓存**，前后端分离。
* 提供冗余，queue里的数据支持备份，可恢复
* 扩展性
* 作为缓冲
* 顺序保证
* 异步通信
    ![HBASE Map Reduce](img/mq_log.png)

#### 消息队列模式
* 点对点 push-pull
    * 缺点，得老有一个进程去pull
* 发布/订阅 push-push


### 4.1 Kafka
* Kafka基于发布/订阅模式
* 发布订阅系统中会有一个broker，也是发布消息的中心点
* scala写的
* kafka的集群管理依赖zookeeper，即broker管理
* 基于topic管理数据

#### 架构
* 主要是producer，kafka集群，consumer
* 每个kafka的高可用，是leader/follower模式
* 和rabbitmq不同，每个consumer有自己的queue，数据实际存在queue中，kafka的数据存在partition里，每个consumer有自己的offset
* consumer, kafka集群 之间的交互通过zookeeper实现
    * producer不和zookeeper打交道

###### topic
* 每个消息对应一个topic，每个topic有多个partition
* partition，分区用来做每个kafka服务器的负载均衡
* 分区在集群中的多个服务器中都有副本，用leader/follower模式
    每个partition i 都有自己的leader和follower，副本必须跨服务器

###### consumer 
* consumer group
    * 同一个consumer group里的consumer不能消费同一个分区的数据
    * 所以他们不会拿到一样的message，可以达到分流的目的

### 安装与配置
#### 集群

* server.properties

```shell
broker.id=0

delete.topc.enable=true

log.dirs=/opt/kafka

num.partitions=1

zookeeper.connet=hadoop102:2181, hadoop103:2181, hadoop104:2181
```

#### 命令
* kafka-topics.sh
* kafka-console-consumer.sh
* kafka-console-producer.sh


```shell
# 启动zookeeper
zkstart.sh

zkstart.sh status

# start kafka, 只能单机一台一台启动
kafka-server.start.sh config/server.properties

# 查看启动的进程
jps -l 

# 创建一个topic，设定需要多少个分区，多少个副本，连接在那个zookeeper端口，名字
kafka-topics.sh --create --zookeeper hadoop102:2181 --partitions 2 --replication-facotr 2 --topic firstTopic 

# 查看topic
kafka-topics.sh --list --zookeeper hadoop102:2181 
```

* produce
```
```

* consume
```shell
# consumer和zookeeper连接
# 每个kafka的offset本来存在broker里，但是这就需要consumer与kafka broker也建立连接，浪费
# 新版本的kafka把offset存在zookeeper中
kafka-console-consumer.sh --zookeeper hadoop102:2191 --topic firstTopic --from-beginning

```
![HBASE Map Reduce](img/kafka.png)

https://www.jianshu.com/p/d3e963ff8b70


https://www.bilibili.com/video/av35354301/?p=2

### 4.2 RabbitMQ
6种模式 https://www.bilibili.com/video/av48444262/?p=4

https://www.cnblogs.com/diegodu/p/4971586.html

* erlang写的

#### 4.2.0 原理
##### ACK与消息持久化
消息应答
* auto ack = true：阅后即焚，消息给到消费者了，MQ就立马把消息从内存中删除
    * 如果消费者执行中宕机，那么会丢失正在处理中的消息
* auto ack = false: 手动模式
    * 如果有一个消费者挂掉，MQ发现有消息没有被ACK，则把该消息给其他消费者
    * 消费者ACK之后，MQ再把消息从内存中删除

**就安全么，内存中的消息，MQ挂掉也就没了，并没有被持久化**
* MQ支持消息的持久化
    * durable=ture，声明队列的时候可以设置该参数，使消息持久化到磁盘
    * 注意声明队列的参数一经设定，不能被修改

##### 交换机/转发器，exhange
* 用法见4.2.3
* 接收producer消息，然后给队列转发
* 交换机不能 persistent 消息
* type
    * exhange = '' 匿名转发
* exhange有自己的分发模式
    * fanout 不处理路由键
        * 只要跟exhange绑定的队列，都能收到消息
    * topic 将路由和某种模式绑定
    * direct 处理路由键
        * routing模式采用该方式
        * 发送的时候有一个路由key，队列也有key，如果match才转发
    
##### 消息确认机制 (事务 + confirm)
* 问题：生产者将消息发送到MQ之后，到底有没有到达
    * 默认是不知道的，MQ不返回任何信息
    * 两种方式
        * AMQP模式，AMQP实现了事务机制
        * confirm模式
        
###### AMQP 事务机制
* txSelect 用于将当前channel设置为transaction模式
* txCommit 用于提交事务
* txRollback 用于回滚事务

* publisher
```java
try {
    channel.txSelect();     //选择Transaction模式
    channel.basicPublish("", "hello", null, "Hello World!");
    channel.txCommit();       //顺利无异常执行结束，则commit
} catch (Exception e) {
    cannel.txRollback();      // 有异常则rollback
}
```

* 缺点
    * 网络IO高，一个操作，需要select，commit，rollback 2-3次IO
    * 降低了吞吐量
           
###### confirm 机制
* 进入confirm模式之后，每一条消息都会有一个ID
* broker在确认消息进入队列后，会返回给生产者一个ACK + message ID
* confirm模式下，是**异步**的
    * Nack, negetive ack, 如果MQ出异常崩溃，则会给producer返回一个Nack, 生产者也有回掉函数来处理
* 编程模式
    * 普通模式 Blocking waitForConfirms()
    * 批量发 Blocking waitForConfirms()
    * Async, 给broker一个callback

* 普通模式

```java
channel.confirm.Select();      //生产者将channel设置为confirm模式

channel.basicPublish("", "hello", null, "Hello World!");

if(!channel.waitForConfirms()){         // Blocking等待confirm
    System.out.println("Message send failure");
}else{
    System.out.println("Message send success");
}
```

* 批量模式

```java
channel.confirm.Select();      //生产者将channel设置为confirm模式

for(int i=0; i<10; i++){
    channel.basicPublish("", "hello", null, "Hello World!");
}

if(!channel.waitForConfirms()){         // Blocking等待confirm
    System.out.println("Message send failure");
}else{
    System.out.println("Message send success");
}
```


#### 4.2.1 简单队列 Simple Queue
##### 模型
![HBASE Map Reduce](img/rabbitmq_1.png)

##### 编程
    * 首先获取和消息中间件，MQ，的一个链接
    * 创建channel通道

###### producer
```python
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")
channel.close()
```

###### consumer

```python
channel.queue_declare(queue='hello')

def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    
channel.basic_consume(queue='hello',
                      auto_ack=True,
                      on_message_callback=callback)

print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()
```


###### 不足
* producer一一对应consumer，耦合度高
    * 如果多个consumer对应一个producer，不行
* 如果队列名变更
    * 生产者，消费者都得更改

#### 4.2.2 work queue 工作队列
一个producer对应多个consumer
##### 模型
![HBASE Map Reduce](img/rabbitmq_2.png)
    * 原因
        * 一般生产者产生message毫不费力
        * 而消费者拿到message之后是需要处理，一般较慢
        * 一对一的关系，会造成queue的积压
        
##### 程序

###### producer
```python
channel.queue_declare(queue='task_queue', durable=True)  # 让消息可以被持久化

message = ' '.join(sys.argv[1:]) or "Hello World!"
channel.basic_publish(
    exchange='',
    routing_key='task_queue',
    body=message,
    properties=pika.BasicProperties(
        delivery_mode=2,  # make message persistent!
    ))
print(" [x] Sent %r" % message)
connection.close()
```

###### consumer
```python
channel.queue_declare(queue='task_queue', durable=True)
print(' [*] Waiting for messages. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    ch.basic_ack(delivery_tag=method.delivery_tag)


channel.basic_qos(prefetch_count=1)   # 确认消息分发方式：公平分发
channel.basic_consume(queue='task_queue', on_message_callback=callback)

channel.start_consuming()
```
###### consumer之间消息怎样dispatch
    * round-robin 轮询分发 - default
        * By default, RabbitMQ will send each message to the next consumer, in sequence.
        * On average every consumer will get the same number of messages. 
        * This way of distributing messages is called round-robin.
    * fair dispatch 公平. basic_qos
        * 限制消费者每次只处理一条消息，直到消费者ack，MQ不会继续给该consumer分发消息
        * 要使用fair dispatch必须disable auto ack，不然没效果嘛


```shell
python worker.py  ## start one consumer

python worker.py ## start another

# default round robin, 也可以使用fair dispatch，把消息在两个consumer之间分发
```

#### 4.2.3 publish/subscribe
##### 模型
![HBASE Map Reduce](img/rabbitmq_3.png)

* 一个Producer，多个consumer
* 每个consumer都有自己的队列
* producer没有直接把消息发送到队列，而是发给交换机，exhange
* 每个队列都要绑定到exchange，才能接受消息

##### 代码

###### producer

```python
# 绑定到exchange上，而不是把channel直接连接到queue
channel.exchange_declare(exchange='logs', exchange_type='fanout') 

message = ' '.join(sys.argv[1:]) or "info: Hello World!"
channel.basic_publish(exchange='logs', routing_key='', body=message)
print(" [x] Sent %r" % message)
connection.close()
```

###### consumer

```python
channel.exchange_declare(exchange='logs', exchange_type='fanout')

# declare自己的queue上
result = channel.queue_declare('', exclusive=True)
queue_name = result.method.queue

# 把queue bind到exchange上
channel.queue_bind(exchange='logs', queue=queue_name)

print(' [*] Waiting for logs. To exit press CTRL+C')

def callback(ch, method, properties, body):
    print(" [x] %r" % body)

channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()
```

#### 4.2.4 routing
##### 模型
![HBASE Map Reduce](img/rabbitmq_4.png)


###### producer

```python
# exhange 设置为direct, channel绑定到exchange
channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

severity = sys.argv[1] if len(sys.argv) > 1 else 'info'
message = ' '.join(sys.argv[2:]) or 'Hello World!'
channel.basic_publish(
    exchange='direct_logs', routing_key=severity, body=message)
print(" [x] Sent %r:%r" % (severity, message))
connection.close()
```

###### consumer

```python
channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

result = channel.queue_declare('', exclusive=True)
queue_name = result.method.queue

severities = ["info", "warning", "error"]

# 需要指定routing_key, 绑定三个routing key到该队列
for severity in severities:
    channel.queue_bind(
        exchange='direct_logs', queue=queue_name, routing_key=severity)

print(' [*] Waiting for logs. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))


channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()
```

#### 4.2.5 Topics
##### 模型
![HBASE Map Reduce](img/rabbitmq_5.png)

* 类似于routing，不过时key matching 支持通配符


#### 4.2.6 RPC
##### 模型
![HBASE Map Reduce](img/rabbitmq_6.png)
###### server
```python
connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='rpc_queue')

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,    # reply is direct mode, send to the requested client
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()
```
###### client
```python
class FibonacciRpcClient(object):

    def __init__(self):
        self.connection = pika.BlockingConnection(
            pika.ConnectionParameters(host='localhost'))

        self.channel = self.connection.channel()

        result = self.channel.queue_declare('', exclusive=True)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(
            queue=self.callback_queue,
            on_message_callback=self.on_response,
            auto_ack=True)

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4())
        self.channel.basic_publish(
            exchange='',
            routing_key='rpc_queue',
            # in the request set a correlation_id, so server know who to send to
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,               
            ),
            body=str(n))
        while self.response is None:
            self.connection.process_data_events()
        return int(self.response)


fibonacci_rpc = FibonacciRpcClient()

print(" [x] Requesting fib(30)")
response = fibonacci_rpc.call(30)
print(" [.] Got %r" % response)
```

### 高可用
* 集群

### 安全
* SSL连接